In [0]:
# Importa o módulo de funções do PySpark para manipulação de colunas e dados
from pyspark.sql import functions as F

In [0]:
# Define o fuso horário da sessão Spark para "America/Sao_Paulo"
spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")

In [0]:
# Caminho da tabela Delta de origem no nível 'silver' do pipeline de dados
origem_path = "fomacao_microsoft_power_bi_profisional.silver.fato_vendas"

In [0]:
# Lê a tabela Delta de origem e seleciona colunas relevantes para o DataFrame 'df_gold'
df_gold = (spark.read.table(origem_path)
           .select(
                 "ProductID"      # Identificador do produto
               , "Date"           # Data da venda
               , "Zip"            # Código postal
               , "Units"          # Quantidade vendida
               , "Revenue"        # Receita gerada
               , "Country"        # País da venda
               , "_ingest_ts_utc" # Timestamp de ingestão em UTC
               , "_ingest_date"   # Data de ingestão
               )
           )

In [0]:
# Nome da tabela de destino no nível 'gold' do pipeline de dados
tbl = 'fato_vendas'
destino_path = f'fomacao_microsoft_power_bi_profisional.gold.{tbl}'

# Escreve o DataFrame 'df_gold' como uma tabela Delta particionada por '_ingest_date'
df_gold = (df_gold.write
             .format('delta')                # Formato Delta Lake
             .mode('overwrite')              # Sobrescreve dados existentes
             .option('overwriteSchema', 'true') # Sobrescreve o schema se necessário
             .partitionBy('_ingest_date')    # Particiona por data de ingestão
             .saveAsTable(destino_path))     # Salva como tabela Delta

# Exibe a contagem de registros da tabela de destino
print(f'Contagem de registros: {spark.table(destino_path).count()}')